In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

from sklearn.neural_network import MLPClassifier

from helpers.plot import group_box_plots, group_roc_curves
from helpers.fairness_measures import *

from aif360.datasets import StandardDataset
from aif360.algorithms.inprocessing import PrejudiceRemover

## Load data

In [ ]:
train = pd.read_csv("/project/data/adult/processed/train-one-hot.csv").sample(
    2000
)
test = pd.read_csv("/project/data/adult/processed/test-one-hot.csv").sample(
    2000
)
val = pd.read_csv("/project/data/adult/processed/val-one-hot.csv").sample(6000)

In [ ]:
train_sds = StandardDataset(
    train,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
test_sds = StandardDataset(
    test,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
val_sds = StandardDataset(
    val,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)

## Perform intervention

In [ ]:
sensitive_attr = "sex"
class_attr = "salary"

In [ ]:
PR = PrejudiceRemover(
    eta=5.0, sensitive_attr=sensitive_attr, class_attr=class_attr
)
PR.fit(train_sds)

In [ ]:
val_sds_pred = PR.predict(val_sds)
val_scores = val_sds_pred.scores.flatten()

In [ ]:
print("Accuracy =", accuracy(val_scores, val.salary))
print(
    "Female accuracy =",
    accuracy(val_scores[val.sex == 0], val.salary[val.sex == 0]),
)
print(
    "Male accuracy =",
    accuracy(val_scores[val.sex == 1], val.salary[val.sex == 1]),
)
print("Mean female score =", val_scores[val.sex == 0].mean())
print("Mean male score =", val_scores[val.sex == 1].mean())

### Plots

In [ ]:
go.Figure(
    data=[
        go.Box(
            x=[sex] * (val.sex == sex).sum(),
            y=val_scores[val.sex == sex],
            name="Male" if sex else "Female",
        )
        for sex in range(2)
    ]
)

In [ ]:
go.Figure(
    data=[
        go.Bar(
            x=[sex],
            y=[val_scores[val.sex == sex].mean()],
            name="Male" if sex else "Female",
        )
        for sex in range(2)
    ]
)